# 2º Atividade
## Transformações de espaços de cores

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
filename = "../content/starryNight.jpg"
img = cv.imread(filename,1)

b, g, r = cv.split(img)
img_rgb = cv.merge([r, g, b])
plt.imshow(img_rgb)

ValueError: not enough values to unpack (expected 3, got 0)

## 1. Implemente uma função que converte uma imagem colorida no padrão RGB para o padrão HSV. Exiba as imagens monocromáticas dos três canais resultantes da conversão.

In [ ]:
def rgb_to_hsv(image):
    # Get the dimensions of the image
    # image_np = np.array(image)
    rows, cols, channels = image.shape
    hsv = np.zeros_like(image, dtype=np.float32) # Initialize with float32 for HSV values
    for i in range(rows):
        for j in range(cols):
            r, g, b = image[i, j, 0], image[i, j, 1], image[i, j, 2] # Extract individual color channels

            # Normalize R, G and B
            r /= 255.0
            g /= 255.0
            b /= 255.0

            max_val = max(r, g, b)
            min_val = min(r, g, b)
            delta = max_val - min_val

            if max_val == min_val:
                h = 0
            elif max_val == r:
                h = (60 * ((g - b) / delta))
            elif max_val == g:
                h = (60 * (((b - r) / delta) + 2))
            elif max_val == b:
                h = (60 * (((r - g) / delta) + 4))

            if h < 0:
                h += 360

            s = 0
            if max_val != 0:
                s = (delta / max_val)

            v = max_val
            hsv[i, j, :] = (h/2, s, v) # Assign HSV values to the pixel

    return hsv

In [ ]:
img_hsv = rgb_to_hsv(img_rgb)
img_hsv_normalized = img_hsv.astype(np.float32) / np.array([2*np.pi, 1, 1], dtype=np.float32)  # Normalize HSV values
plt.imshow(img_hsv_normalized)

In [ ]:
hue, saturation, value = cv.split(img_hsv_normalized)

fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(131)
ax1.set_title('Matiz')
plt.imshow(hue, cmap='gray')

ax2 = fig.add_subplot(132)
ax2.set_title('Saturação')
plt.imshow(saturation, cmap='gray')

ax3 = fig.add_subplot(133)
ax3.set_title('Valor')
plt.imshow(value, cmap='gray')

plt.show()

## 2. Implemente uma função que converte uma imagem colorida no padrão HSV para o padrão RGB. Exiba as imagens monocromáticas dos três canais resultantes da conversão. Compare o resultado com as imagens monocromáticas dos canais da imagem RGB original.

In [ ]:
# When 0 ≤ H < 360, 0 ≤ S and V ≤ 1:

def hsv_to_rgb(image):
    rows, cols, channels = image.shape
    rgb = np.zeros_like(image, dtype=np.uint8)
    for i in range(rows):
        for j in range(cols):
            h, s, v = image[i, j, 0], image[i, j, 1], image[i, j, 2]

            c = v * s
            x = (1 - abs((h / 60) % 2 - 1)) * c
            m = v - c

            if 0 <= h < 60:
                r = c
                g = x
                b = 0
            elif 60 <= h < 120:
                r = x
                g = c
                b = 0
            elif 120 <= h < 180:
                r = 0
                g = c
                b = x
            elif 180 <= h < 240:
                r = 0
                g = x
                b = c
            elif 240 <= h < 300:
                r = x
                g = 0
                b = c
            elif 300 <= h < 360:
                r = c
                g = 0
                b = x

            rgb[i, j, 0] = int(r + m) * 255
            rgb[i, j, 1] = int(g + m) * 255
            rgb[i, j, 2] = int(b + m) * 255

    return rgb

In [ ]:
# When 0 ≤ H < 360, 0 ≤ S and V ≤ 1:

def hsv_to_rgb(image):
    rows, cols, channels = image.shape
    rgb = np.zeros_like(image, dtype=np.uint8)
    for i in range(rows):
        for j in range(cols):


            hi = np.floor(image[..., 0] / 60.0) % 6
            hi = hi.astype('uint8')
            v = image[..., 2].astype('float')
            f = (image[..., 0] / 60.0) - np.floor(image[..., 0] / 60.0)
            p = v * (1.0 - image[..., 1])
            q = v * (1.0 - (f * image[..., 1]))
            t = v * (1.0 - ((1.0 - f) * image[..., 1]))

            rgb[hi == 0, :] = np.dstack((v, t, p))[hi == 0, :]
            rgb[hi == 1, :] = np.dstack((q, v, p))[hi == 1, :]
            rgb[hi == 2, :] = np.dstack((p, v, t))[hi == 2, :]
            rgb[hi == 3, :] = np.dstack((p, q, v))[hi == 3, :]
            rgb[hi == 4, :] = np.dstack((t, p, v))[hi == 4, :]
            rgb[hi == 5, :] = np.dstack((v, p, q))[hi == 5, :]

    return rgb

In [ ]:
scalar = float # a scale value (0.0 to 1.0)
def priv_hsv_to_rgb( h:scalar, s:scalar, v:scalar) -> tuple:
    if s:
        if h == 1.0: h = 0.0
        i = int(h*6.0)
        f = h*6.0 - i

        w = v * (1.0 - s)
        q = v * (1.0 - s * f)
        t = v * (1.0 - s * (1.0 - f))

        if i==0: return (v, t, w)
        if i==1: return (q, v, w)
        if i==2: return (w, v, t)
        if i==3: return (w, q, v)
        if i==4: return (t, w, v)
        if i==5: return (v, w, q)
        else: return (v, w, q)
    else: return (v, v, v)

def hsv_2_rgb_pixel( h:scalar, s:scalar, v:scalar) -> tuple:
    ret = priv_hsv_to_rgb(h.astype(scalar), s.astype(scalar), v.astype(scalar))
    #return list(map(lambda x: int(min(round(x*255), 255)), ret))
    return ret

In [ ]:
def hsv_2_rgb_img(img: np.ndarray) -> np.ndarray:
    conv = np.copy(img)

    for lines in range(len(img)):
        for col in range(len(img[lines])):
            conv[lines][col] = np.array(hsv_2_rgb_pixel(img[lines][col][0], img[lines][col][1], img[lines][col][2]))

    return conv

Como os valores HSV são normalmente representados em intervalos diferentes
(Matiz: 0-360, Saturação e Valor: 0-1),
exibir diretamente a matriz img_hsv pode levar a resultados inesperados.
Por isso, necessário uma normalização para exibir com o matplotlib

In [ ]:
print(img_hsv_normalized)

In [ ]:
img_rgb_from_hsv = hsv_2_rgb_img(img_hsv_normalized)
plt.imshow(img_rgb_from_hsv)

In [ ]:
n_r, n_g, n_b = cv.split(img_rgb_from_hsv)

In [ ]:
# Create a figure with three subplots
fig = plt.figure(figsize=(15, 5))

# Subplot 1: Red channel
ax1 = fig.add_subplot(131)
ax1.set_title('Red Channel (gray)')
plt.imshow(n_r, cmap='gray')

# Subplot 2: Green channel
ax2 = fig.add_subplot(132)
ax2.set_title('Green Channel (gray)')
plt.imshow(n_g, cmap='gray')

# Subplot 3: Blue channel
ax3 = fig.add_subplot(133)
ax3.set_title('Blue Channel (gray)')
plt.imshow(n_b, cmap='gray')

# Show the figure
plt.show()

In [ ]:
fig1 = plt.figure(1, figsize=(20, 9))
# create subplot in fig1
ax1 = fig1.subplots(2,3)
# set 1st collumn plot
ax1[0][0].imshow(r, cmap='Reds')
ax1[0][0].title.set_text('Red Channel (cvt)')
ax1[0][0].set_xticks([])
ax1[0][0].set_yticks([])

ax1[1][0].imshow(n_r, cmap='Reds')
ax1[1][0].title.set_text('Red Channel')
ax1[1][0].set_xticks([])
ax1[1][0].set_yticks([])

# set 2nd collumn plot
ax1[0][1].imshow(g, cmap='Greens')
ax1[0][1].title.set_text('Green Channel (cvt)')
ax1[0][1].set_xticks([])
ax1[0][1].set_yticks([])

ax1[1][1].imshow(n_g, cmap='Greens')
ax1[1][1].title.set_text('Green Channel')
ax1[1][1].set_xticks([])
ax1[1][1].set_yticks([])
#3
ax1[0][2].imshow(b, cmap='Blues')
ax1[0][2].title.set_text('Blue Channel (cvt)')
ax1[0][2].set_xticks([])
ax1[0][2].set_yticks([])

ax1[1][2].imshow(n_b, cmap='Blues')
ax1[1][2].title.set_text('Blue Channel')
ax1[1][2].set_xticks([])
ax1[1][2].set_yticks([])
# show figures
plt.show()

## 3. Caso sejam observadas diferenças entre os canais RGB originais e os canais RGB reconvertidos pela função que você implementou, escreva um curto parágrafo justificando estas diferenças.

As diferenças entre as imagens são claras, durante a conversão há algumas conversões diretas onde há perda de informação, por isso, as imagens convertidas não voltam a ser as mesmas.

## 4. Compare os resultados das conversões implementadas manualmente com as conversões utilizando a função cv.cvtColor().

In [ ]:
img_cvtHSV = cv.cvtColor(img_rgb, cv.COLOR_RGB2HSV)
plt.imshow(img_cvtHSV)

In [ ]:
# Create a figure with two subplots
fig = plt.figure(figsize=(15, 5))

# Subplot 1: Hue channel
ax1 = fig.add_subplot(121)
ax1.set_title('Convertion manual RGB to HSV')
plt.imshow(img_hsv_normalized)

ax2 = fig.add_subplot(122)
ax2.set_title('Cvt Convertion RGB to HSV')
plt.imshow(img_cvtHSV)

# Show the figure
plt.show()

In [ ]:
# Create a figure with two subplots
fig = plt.figure(figsize=(15, 5))

# Subplot 1: Hue channel
ax1 = fig.add_subplot(121)
ax1.set_title('Convertion manual HSV to RGB')
plt.imshow(img_rgb_from_hsv)

ax2 = fig.add_subplot(122)
ax2.set_title('Cvt Convertion HSV to RGB')
plt.imshow(cv.cvtColor(img_cvtHSV, cv.COLOR_HSV2RGB))

# Show the figure
plt.show()

## 5. Implemente uma função que converte uma imagem colorida do padrão RGB para uma imagem em escala de cinza. Mostre o resultado da sua conversão e compare com a conversão utilizando cv.cvtColor()

In [ ]:
img_grayscale = cv.imread(filename,0)
plt.imshow(img_grayscale, cmap="gray")

In [ ]:
# Get the individual color channels
cvt_red_channel, cvt_green_channel, cvt_blue_channel = cv.split(img_rgb)

# Create a figure with three subplots
fig = plt.figure(figsize=(15, 5))

# Subplot 1: Red channel
ax1 = fig.add_subplot(131)
ax1.set_title('Red Channel (cvt)')
plt.imshow(cvt_red_channel, cmap='gray')

# Subplot 2: Green channel
ax2 = fig.add_subplot(132)
ax2.set_title('Green Channel (cvt)')
plt.imshow(cvt_green_channel, cmap='gray')

# Subplot 3: Blue channel
ax3 = fig.add_subplot(133)
ax3.set_title('Blue Channel (cvt)')
plt.imshow(cvt_blue_channel, cmap='gray')

# Show the figure
plt.show()


In [ ]:
def RGB_2_gray_img(img: np.ndarray) -> np.ndarray:
    conv = np.copy(img)

    for lines in range(len(img)):
        for col in range(len(img[lines])):
            conv[lines][col] = (lambda r, g, b: np.full(3,r*.299+g*.587+b*.114))(img[lines][col][0], img[lines][col][1], img[lines][col][2])
    return conv

In [ ]:
img_gray_conv = RGB_2_gray_img(img_rgb)
plt.imshow(img_gray_conv)

In [ ]:
# Get the individual color channels
red_channel, green_channel, blue_channel = cv.split(img_gray_conv)

# Create a figure with three subplots
fig = plt.figure(figsize=(15, 5))

# Subplot 1: Red channel
ax1 = fig.add_subplot(131)
ax1.set_title('Red Channel')
plt.imshow(red_channel, cmap='gray')

# Subplot 2: Green channel
ax2 = fig.add_subplot(132)
ax2.set_title('Green Channel')
plt.imshow(green_channel, cmap='gray')

# Subplot 3: Blue channel
ax3 = fig.add_subplot(133)
ax3.set_title('Blue Channel')
plt.imshow(blue_channel, cmap='gray')

# Show the figure
plt.show()

In [ ]:
# Create a figure with two subplots
fig = plt.figure(figsize=(15, 5))

# Subplot 1: Hue channel
ax1 = fig.add_subplot(121)
ax1.set_title('Convertion manual RGB to GRAY')
plt.imshow(img_gray_conv)

ax2 = fig.add_subplot(122)
ax2.set_title('Cvt Convertion RGB to GRAY')
plt.imshow(cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY), cmap='gray')

# Show the figure
plt.show()

In [ ]:
fig1 = plt.figure(1, figsize=(20, 9))
# create subplot in fig1
ax1 = fig1.subplots(2,3)
# set 1st collumn plot
ax1[0][0].imshow(cvt_red_channel, cmap='gray')
ax1[0][0].title.set_text('Red Channel (cvt)')
ax1[0][0].set_xticks([])
ax1[0][0].set_yticks([])

ax1[1][0].imshow(red_channel, cmap='gray')
ax1[1][0].title.set_text('Red Channel')
ax1[1][0].set_xticks([])
ax1[1][0].set_yticks([])

# set 2nd collumn plot
ax1[0][1].imshow(cvt_green_channel, cmap='gray')
ax1[0][1].title.set_text('Green Channel (cvt)')
ax1[0][1].set_xticks([])
ax1[0][1].set_yticks([])

ax1[1][1].imshow(green_channel, cmap='gray')
ax1[1][1].title.set_text('Green Channel')
ax1[1][1].set_xticks([])
ax1[1][1].set_yticks([])
#3
ax1[0][2].imshow(cvt_blue_channel, cmap='gray')
ax1[0][2].title.set_text('Blue Channel (cvt)')
ax1[0][2].set_xticks([])
ax1[0][2].set_yticks([])

ax1[1][2].imshow(blue_channel, cmap='gray')
ax1[1][2].title.set_text('Blue Channel')
ax1[1][2].set_xticks([])
ax1[1][2].set_yticks([])
# show figures
plt.show()